In [4]:
import numpy as np
import synthetic_data as sd
import network_builder as nb
import networkx as nx
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from netCDF4 import Dataset

In [48]:
f_pre = Dataset('precipitation.nc')
#f_pre = Dataset('precip.mon.total.v501.nc')

#f_temp = Dataset('temperatureCDC.nc') 
pr = f_pre.variables['precip']
lon = f_pre.variables['lon'][:]
lat = f_pre.variables['lat'][:]
#pr = np.swapaxes(pr,0,2)

In [49]:
lat

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    units: degrees_north
    actual_range: [ 38.75 -38.75]
    long_name: Latitude
    standard_name: latitude
    axis: Y
    coordinate_defines: center
unlimited dimensions: 
current shape = (32,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [44]:
pr

<class 'netCDF4._netCDF4.Variable'>
float32 precip(time, lat, lon)
    long_name: GPCC Monthly Total of precipitation
    units: mm
    missing_value: -9.96921e+36
    dataset: GPCC Precipitation 2.5degree V2018 Full Reanalysis
    actual_range: [  0.   536.17]
    parent_stat: Observations
    var_desc: Precipitation
    level_desc: Surface
    valid_range: [   0. 8000.]
    statistic: Total
    level: Surface
unlimited dimensions: time
current shape = (841, 32, 34)
filling on, default _FillValue of 9.969209968386869e+36 used

In [141]:
def create_data(y, tau):
    #tau = tau + 1
    database = []
    for i in np.arange(0, len(y)):
        l = []
        for t in np.arange(0, tau):
            if (i+t) < len(y):
                a = float(y[i+t])
                l.append(a)
        if(len(l) == tau):
            database.append(np.array(l))
    return np.array(database)

In [216]:
y = pr[18,14,:]

In [217]:
X = create_data(y,3)

In [218]:
Y = y[3:]

In [219]:
X = X[:-1]

In [220]:
p = 0.8
train_size = int(len(X) * p)
test_size = len(X) - train_size

In [221]:
X_train, X_test = X[0:train_size], X[train_size:len(X)]

In [222]:
Y_train, Y_test = Y[0:train_size], Y[train_size:len(X)]

In [56]:
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

In [223]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50,20, ), random_state=1)

In [224]:
clf.fit(X_train, Y_train)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [225]:
y_pred = clf.predict(X_test)

In [226]:
r2_score(Y_test,y_pred)

0.41768776624827475

In [227]:
mean_squared_error(Y_test,y_pred)

2811.5408924283774

In [228]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, Y_train);

In [229]:
y_pred = rf.predict(X_test)

In [230]:
r2_score(Y_test,y_pred)

0.5229802451063303

In [231]:
mean_squared_error(Y_test,y_pred)

2303.1639550810664